### This notebook cleans and preprocesses the WeFarm SMS dataset from Producers Direct - <br> and subsets the **questions** into new datafiles for **live** and **zombie** questioners

### Input (already downloaded):  'b0cd514b-b9cc-4972-a0c2-c91726e6d825.csv'

### Outputs (into working directory): 
* raw dataset: 'new_df.parquet'
* all unique questions:  'questions_df.parquet'
* kenya: 'questions_kenya_df.parquet', 'questions_kenya_df.csv'
* tanzania:  'questions_tanzania_df.parquet', 'questions_tanzania_df.csv'
* uganda:  'questions_uganda_df.parquet', 'questions_uganda_df.csv'
* gambia:  'questions_gambia_df.parquet', 'questions_gambia_df.csv'

### Steps:
* Summarize the data fields
* Clean the dataset by removing fields with null values or only 1 value, duplicate rows, change data types, etc.
* Save processed data into new parquet file
* Subset data by unique questions asked in 4 countries:  Kenya, Tanzania, Uganda, and Gambia
* Save subsets as parquet files


In [2]:
#import packages:  pandas, numpy, fastparquet (for saving lg data files)
import pandas as pd
import numpy as np
import fastparquet as fp

In [8]:
#initial:  read file, will take approx. 15 min
#csv_path = "F:\\datakind-farmer-2025\\b0cd514b-b9cc-4972-a0c2-c91726e6d825.csv"
raw_df = pd.read_csv(csv_path)


### Data Exploration

In [6]:
#see initial records
raw_df.head()

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,...,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,NaN,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',...,ug,NaN,NaN,2017-11-18 13:09:11+00,farmer,live,ug,NaN,NaN,2017-05-09 09:19:33+00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,...,ug,NaN,NaN,2017-11-20 11:55:48+00,farmer,zombie,ug,NaN,NaN,2017-11-22 10:13:03+00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,NaN,NaN,2017-08-22 14:51:07+00,farmer,zombie,ug,NaN,NaN,2017-08-12 09:30:33+00


In [9]:
#exploratory stuff
raw_df.shape


(20304843, 24)

In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20304843 entries, 0 to 20304842
Data columns (total 24 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   question_id                 int64 
 1   question_user_id            int64 
 2   question_language           object
 3   question_content            object
 4   question_topic              object
 5   question_sent               object
 6   response_id                 int64 
 7   response_user_id            int64 
 8   response_language           object
 9   response_content            object
 10  response_topic              object
 11  response_sent               object
 12  question_user_type          object
 13  question_user_status        object
 14  question_user_country_code  object
 15  question_user_gender        object
 16  question_user_dob           object
 17  question_user_created_at    object
 18  response_user_type          object
 19  response_user_status        object
 20  

In [12]:
# count NaNs, see which columns can be dropped
nulls = raw_df.isna().sum()
print(nulls)

question_id                          0
question_user_id                     0
question_language                    0
question_content                     0
question_topic                 3537729
question_sent                        0
response_id                          0
response_user_id                     0
response_language                    0
response_content                     0
response_topic                13835556
response_sent                        0
question_user_type                   0
question_user_status                 0
question_user_country_code           0
question_user_gender          19713950
question_user_dob             19073559
question_user_created_at             0
response_user_type                   0
response_user_status                 0
response_user_country_code           0
response_user_gender          19339833
response_user_dob             18640155
response_user_created_at             0
dtype: int64


In [24]:
category = 'question_language'
unique=raw_df[category].unique()
print(unique)

['nyn' 'eng' 'swa' 'lug']


In [33]:
category = 'question_topic'
unique=raw_df[category].unique()
print(unique)
print(raw_df[category].nunique())

[nan 'cattle' 'cat' 'poultry' 'rabbit' 'pig' 'coconut' 'plant' 'tomato'
 'animal' 'potato' 'watermelon' 'coffee' 'onion' 'chicken' 'rice' 'maize'
 'napier-grass' 'tree' 'sheep' 'passion-fruit' 'cassava' 'pigeon' 'banana'
 'kale' 'bean' 'wheat' 'cereal' 'carrot' 'cabbage' 'crop' 'spinach'
 'turkey' 'peach' 'goat' 'paw-paw' 'butternut-squash' 'acacia' 'livestock'
 'sweet-potato' 'greens' 'pumpkin' 'millet' 'bird' 'plantain' 'olive'
 'vegetable' 'tobacco' 'sugar-cane' 'bee' 'avocado' 'pineapple' 'beetroot'
 'melon' 'dog' 'guava' 'capsicum' 'miraa' 'grass' 'mango' 'macademia'
 'sesame' 'pear' 'jackfruit' 'cowpea' 'nightshade' 'cotton' 'guinea-fowl'
 'flax' 'apple' 'fish' 'tea' 'cocoa' 'garlic' 'duck' 'sunflower' 'orange'
 'peanut' 'soya' 'squash' 'tilapia' 'safflower' 'mushroom'
 'collard-greens' 'french-bean' 'mung-bean' 'lettuce' 'finger-millet'
 'aubergine' 'yam' 'oat' 'clover' 'strawberry' 'parsley' 'pea' 'rapeseed'
 'radish' 'amaranth' 'taro' 'lemon' 'cucumber' 'eucalyptus' 'chilli'
 

In [27]:
category = 'response_user_type'
unique=raw_df[category].unique()
print(unique)


['farmer']


In [ ]:
category = 'question_user_type'
unique=raw_df[category].unique()
print(unique)

In [28]:
category = 'question_user_status'
unique=raw_df[category].unique()
print(unique)

['live' 'zombie' 'destroyed' 'blocked']


In [30]:
category = 'response_user_status'
unique=raw_df[category].unique()
print(unique)

['live' 'zombie' 'destroyed' 'blocked']


In [31]:
category = 'question_user_country_code'
unique=raw_df[category].unique()
print(unique)

['ug' 'ke' 'gb' 'tz']


In [32]:
category = 'response_user_country_code'
unique=raw_df[category].unique()
print(unique)

['ug' 'ke' 'gb' 'tz']


In [ ]:
#calculate # of unique values for question id and user fields
print(raw_df['question_id'].nunique())

In [13]:
print(raw_df['question_user_created_at'].nunique())

1017289


In [14]:
print(raw_df['response_user_created_at'].nunique())

947681


In [35]:
print(raw_df['question_user_id'].nunique())

1026367


In [17]:
print(raw_df['response_user_id'].nunique())

960431


### Create new data file for processed data, save as parquet file

In [18]:
#drop columns with NAs and user creation dates
#col_categories = ['response_topic', 'response_user_type', 'question_user_type', 'question_user_dob', 'response_user_dob','question_user_gender','response_user_gender']
new_df = raw_df.drop(columns=['response_user_created_at','response_user_created_at', 'response_topic', 'response_user_type', 'question_user_type', 'question_user_dob', 'response_user_dob','question_user_gender','response_user_gender'])

print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20304843 entries, 0 to 20304842
Data columns (total 16 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   question_id                 int64 
 1   question_user_id            int64 
 2   question_language           object
 3   question_content            object
 4   question_topic              object
 5   question_sent               object
 6   response_id                 int64 
 7   response_user_id            int64 
 8   response_language           object
 9   response_content            object
 10  response_sent               object
 11  question_user_status        object
 12  question_user_country_code  object
 13  question_user_created_at    object
 14  response_user_status        object
 15  response_user_country_code  object
dtypes: int64(4), object(12)
memory usage: 2.4+ GB
None


In [23]:
#change type of (question_sent,response_sent) to UTC date format, which will be preserved in parquet but not CSV formats:
new_df['question_sent_date']=pd.to_datetime(new_df['question_sent'],format='ISO8601')
new_df['response_sent_date']=pd.to_datetime(new_df['response_sent'],format='ISO8601')
new_df.columns

Index(['question_id', 'question_user_id', 'question_language',
       'question_content', 'question_topic', 'question_sent', 'response_id',
       'response_user_id', 'response_language', 'response_content',
       'response_sent', 'question_user_status', 'question_user_country_code',
       'question_user_created_at', 'response_user_status',
       'response_user_country_code', 'question_sent_date',
       'response_sent_date'],
      dtype='object')

In [24]:
new_df.head()

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,response_sent,question_user_status,question_user_country_code,question_user_created_at,response_user_status,response_user_country_code,question_sent_date,response_sent_date
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,NaN,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',2019-01-24 17:54:06.216221+00,live,ug,2017-11-18 13:09:11+00,live,ug,2017-11-22 12:25:03+00:00,2019-01-24 17:54:06.216221+00:00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,2018-01-04 08:57:28+00,live,ug,2017-11-20 11:55:48+00,zombie,ug,2017-11-22 12:25:05+00:00,2018-01-04 08:57:28+00:00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,2017-11-22 12:35:26+00,zombie,ug,2017-08-22 14:51:07+00,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,2017-11-22 12:35:26+00,zombie,ug,2017-08-22 14:51:07+00,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,2017-11-22 12:35:26+00,zombie,ug,2017-08-22 14:51:07+00,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00


In [25]:
#drop old date strings
new_df = new_df.drop(columns=['question_sent', 'response_sent'])
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20304843 entries, 0 to 20304842
Data columns (total 16 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 int64              
 1   question_user_id            int64              
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_id                 int64              
 6   response_user_id            int64              
 7   response_language           object             
 8   response_content            object             
 9   question_user_status        object             
 10  question_user_country_code  object             
 11  question_user_created_at    object             
 12  response_user_status        object             
 13  response_user_country_code  object             
 14  question_sent_date          date

In [26]:
#forgot to drop question user create date
new_df = new_df.drop(columns=['question_user_created_at'])
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20304843 entries, 0 to 20304842
Data columns (total 15 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 int64              
 1   question_user_id            int64              
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_id                 int64              
 6   response_user_id            int64              
 7   response_language           object             
 8   response_content            object             
 9   question_user_status        object             
 10  question_user_country_code  object             
 11  response_user_status        object             
 12  response_user_country_code  object             
 13  question_sent_date          datetime64[ns, UTC]
 14  response_sent_date          date

In [27]:
#change type(question id,user id) to strings
new_df['question_id'] = new_df['question_id'].astype(str)
new_df['response_id'] = new_df['response_id'].astype(str)
new_df['question_user_id'] = new_df['question_user_id'].astype(str)
new_df['response_user_id'] = new_df['response_user_id'].astype(str)
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20304843 entries, 0 to 20304842
Data columns (total 15 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 object             
 1   question_user_id            object             
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_id                 object             
 6   response_user_id            object             
 7   response_language           object             
 8   response_content            object             
 9   question_user_status        object             
 10  question_user_country_code  object             
 11  response_user_status        object             
 12  response_user_country_code  object             
 13  question_sent_date          datetime64[ns, UTC]
 14  response_sent_date          date

In [29]:
#count occurences of unique values in:language, country, topic, user status
#num_language = new_df['question_language'].value_counts()
print("The count by question language:", new_df['question_language'].value_counts())
#num_language = new_df['question_language'].value_counts()
print("The count by user question country", new_df['question_user_country_code'].value_counts())
print("The count by topic", new_df['question_topic'].value_counts())

The count by question language: question_language
eng    11976781
swa     6480848
nyn     1157228
lug      689986
Name: count, dtype: int64
The count by user question country question_user_country_code
ke    9758607
ug    6312194
tz    4233726
gb        316
Name: count, dtype: int64
The count by topic question_topic
maize           2201755
cattle          1675473
chicken         1667251
tomato          1096386
poultry         1051797
                 ...   
setaria              94
mulberry             55
blackberry           47
purple-vetch         34
cranberry             6
Name: count, Length: 148, dtype: int64


In [32]:
print("The count by question user status", new_df['question_user_status'].value_counts())
print("The count by response user status", new_df['response_user_status'].value_counts())

The count by question user status question_user_status
live         13012402
zombie        4175196
destroyed     1630843
blocked       1486402
Name: count, dtype: int64
The count by response user status response_user_status
live         13725431
zombie        3235883
destroyed     2173801
blocked       1169728
Name: count, dtype: int64


In [34]:
#save to new format for faster reloading
new_df.to_parquet("new_df.parquet")

### parquet file was read at later date for more processing, e.g. creating datafile of unique questions

In [6]:
#afterwards:  load processed parquet file
new_df = pd.read_parquet("new_df.parquet")


In [8]:
new_df.head()

,question_id,question_user_id,question_language,question_content,question_topic,response_id,response_user_id,response_language,response_content,question_user_status,question_user_country_code,response_user_status,response_user_country_code,question_sent_date,response_sent_date
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',live,ug,live,ug,2017-11-22 12:25:03+00:00,2019-01-24 17:54:06.216221+00:00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,4334249,526113,eng,Q1 which stage is marleks last vaccinated,live,ug,zombie,ug,2017-11-22 12:25:05+00:00,2018-01-04 08:57:28+00:00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,zombie,ug,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,zombie,ug,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,zombie,ug,zombie,ug,2017-11-22 12:25:08+00:00,2017-11-22 12:35:26+00:00


In [11]:
#create new db of unique questions:
#step 1:  drop duplicates by keeping first instance -> new df
#step 2:  drop response data fields
#step2:  count # responses by question -> new df
#step 3:  merge the two dfs
question_df = new_df.drop_duplicates(subset=['question_id'], keep='first')
print(question_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5865819 entries, 0 to 20304842
Data columns (total 15 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 object             
 1   question_user_id            object             
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_id                 object             
 6   response_user_id            object             
 7   response_language           object             
 8   response_content            object             
 9   question_user_status        object             
 10  question_user_country_code  object             
 11  response_user_status        object             
 12  response_user_country_code  object             
 13  question_sent_date          datetime64[ns, UTC]
 14  response_sent_date          datetime64

### Add # responses for each question to the new db:

In [13]:
#drop response data, except for response language...for now
question_df = question_df.drop(columns=['response_id','response_user_id', 'response_content', 'response_user_status','response_sent_date'])
question_df.head()

,question_id,question_user_id,question_language,question_content,question_topic,response_language,question_user_status,question_user_country_code,response_user_country_code,question_sent_date
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,nyn,live,ug,ug,2017-11-22 12:25:03+00:00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,eng,live,ug,ug,2017-11-22 12:25:05+00:00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,nyn,zombie,ug,ug,2017-11-22 12:25:08+00:00
6,3849078,174909,nyn,E. Radio ezimwagaba nituzituga tunta ariho aba...,None,nyn,zombie,ug,ug,2017-11-22 12:25:09+00:00
7,3849082,417525,swa,S dawa ya viroboto.kwa kuku,poultry,swa,live,ke,ke,2017-11-22 12:25:10+00:00


In [25]:
#count # responses and responders
subset_df = new_df[['question_id','response_id','response_user_id']]



In [26]:
print(subset_df.head())

  question_id response_id response_user_id
0     3849056    20691011           200868
1     3849061     4334249           526113
2     3849077     3849291           296187
3     3849077     3849291           296187
4     3849077     3849291           296187


In [27]:
id_group = subset_df.groupby(['question_id'])
response_group = id_group.count()

In [28]:
print(response_group.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5865819 entries, 10000005 to 9999980
Data columns (total 2 columns):
 #   Column            Dtype
---  ------            -----
 0   response_id       int64
 1   response_user_id  int64
dtypes: int64(2)
memory usage: 134.3+ MB
None


In [29]:
print(response_group.head())

             response_id  response_user_id
question_id                               
10000005               1                 1
10000023               3                 3
10000024               2                 2
10000045               5                 5
10000046               4                 4


In [34]:
response_group.shape

(5865819, 2)

In [39]:
#merge response counts into question db:
question_merge_df = pd.merge(question_df, response_group, on='question_id', how='left')
question_merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865819 entries, 0 to 5865818
Data columns (total 12 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 object             
 1   question_user_id            object             
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_language           object             
 6   question_user_status        object             
 7   question_user_country_code  object             
 8   response_user_country_code  object             
 9   question_sent_date          datetime64[ns, UTC]
 10  response_id                 int64              
 11  response_user_id            int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(9)
memory usage: 537.0+ MB


In [41]:
response_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5865819 entries, 10000005 to 9999980
Data columns (total 2 columns):
 #   Column            Dtype
---  ------            -----
 0   response_id       int64
 1   response_user_id  int64
dtypes: int64(2)
memory usage: 134.3+ MB


In [42]:
question_merge_df.head()


,question_id,question_user_id,question_language,question_content,question_topic,response_language,question_user_status,question_user_country_code,response_user_country_code,question_sent_date,response_id,response_user_id
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,nyn,live,ug,ug,2017-11-22 12:25:03+00:00,1,1
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,eng,live,ug,ug,2017-11-22 12:25:05+00:00,1,1
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,nyn,zombie,ug,ug,2017-11-22 12:25:08+00:00,4,4
3,3849078,174909,nyn,E. Radio ezimwagaba nituzituga tunta ariho aba...,None,nyn,zombie,ug,ug,2017-11-22 12:25:09+00:00,1,1
4,3849082,417525,swa,S dawa ya viroboto.kwa kuku,poultry,swa,live,ke,ke,2017-11-22 12:25:10+00:00,2,2


In [47]:
question_merge_df.columns

Index(['question_id', 'question_user_id', 'question_language',
       'question_content', 'question_topic', 'response_language',
       'question_user_status', 'question_user_country_code',
       'response_user_country_code', 'question_sent_date', 'response_num',
       'response_user_id'],
      dtype='object')

In [45]:
question_merge_df.rename(columns={'response_id': 'response_num'}, inplace=True)

In [48]:
question_merge_df.rename(columns={'response_num': 'response_count'}, inplace=True)

In [49]:
question_merge_df.columns

Index(['question_id', 'question_user_id', 'question_language',
       'question_content', 'question_topic', 'response_language',
       'question_user_status', 'question_user_country_code',
       'response_user_country_code', 'question_sent_date', 'response_count',
       'response_user_id'],
      dtype='object')

In [50]:
#drop response user count, because it's the same as response count and save as csv, ~ 800 mb
Q_df = question_merge_df.drop(columns=['response_user_id'])
#Q_df.to_csv('questions_df.csv', index=False)

### More data exploration...

In [53]:
#count occurences of unique values in:language, country, topic, user status
#num_language = Q_df['question_language'].value_counts()
print("The count by user languages:", Q_df['question_language'].value_counts())
print("The count by user question country", Q_df['question_user_country_code'].value_counts())
print("The count by topic", Q_df['question_topic'].value_counts())
print("The count by question user status", Q_df['question_user_status'].value_counts())


The count by user languages: question_language
eng    2940542
swa    2225974
nyn     433345
lug     265958
Name: count, dtype: int64
The count by user question country question_user_country_code
ke    2422907
ug    1974722
tz    1468072
gb        118
Name: count, dtype: int64
The count by topic question_topic
maize           535567
chicken         430895
cattle          425028
tomato          326011
poultry         272243
                 ...  
leucaena            17
setaria             15
mulberry            13
purple-vetch         7
cranberry            3
Name: count, Length: 148, dtype: int64
The count by question user status question_user_status
live         3830235
zombie       1120098
blocked       481038
destroyed     434448
Name: count, dtype: int64


In [55]:
#blank topics
print(Q_df['question_topic'].isna().sum())

1672009


In [56]:
Q_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865819 entries, 0 to 5865818
Data columns (total 11 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 object             
 1   question_user_id            object             
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_language           object             
 6   question_user_status        object             
 7   question_user_country_code  object             
 8   response_user_country_code  object             
 9   question_sent_date          datetime64[ns, UTC]
 10  response_count              int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(9)
memory usage: 492.3+ MB


In [18]:
Q_df['question_sent_date'] = pd.to_datetime(Q_df['question_sent_date'],format='ISO8601')
Q_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865819 entries, 0 to 5865818
Data columns (total 11 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 int64              
 1   question_user_id            int64              
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   response_language           object             
 6   question_user_status        object             
 7   question_user_country_code  object             
 8   response_user_country_code  object             
 9   question_sent_date          datetime64[ns, UTC]
 10  response_count              int64              
dtypes: datetime64[ns, UTC](1), int64(3), object(7)
memory usage: 492.3+ MB


In [12]:
#check if question  == response language
print("question = response language: ",Q_df['question_language'].equals(Q_df['response_language']))
#check if question  == response country
print("question = response country: ",Q_df['question_user_country_code'].equals(Q_df['response_user_country_code']))


question = response language:  False
question = response country:  False


In [19]:
#drop response_language & response_user_country_code
Q_df = Q_df.drop(columns=['response_language', 'response_user_country_code'])
Q_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865819 entries, 0 to 5865818
Data columns (total 9 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 int64              
 1   question_user_id            int64              
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   question_user_status        object             
 6   question_user_country_code  object             
 7   question_sent_date          datetime64[ns, UTC]
 8   response_count              int64              
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 402.8+ MB


In [21]:
#write to cvs & parquet files before subsetting questions based on country & question user status
Q_df.to_csv('questions_df.csv', index=False)
Q_df.to_parquet("questions_df.parquet")

In [26]:
# subset kenya datasets
Q_kenya_df = Q_df[(Q_df['question_user_country_code'] == 'ke') & ((Q_df['question_user_status'] == 'live') | (Q_df['question_user_status'] == 'zombie'))]

In [32]:
#checking country subset data
Q_kenya_df.info()
Q_kenya_df.head()
print("# of countries: ",Q_kenya_df['question_user_country_code'].nunique())
print("# of unique user statuses: ", Q_kenya_df['question_user_status'].nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 2128052 entries, 4 to 5865818
Data columns (total 9 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   question_id                 int64              
 1   question_user_id            int64              
 2   question_language           object             
 3   question_content            object             
 4   question_topic              object             
 5   question_user_status        object             
 6   question_user_country_code  object             
 7   question_sent_date          datetime64[ns, UTC]
 8   response_count              int64              
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 162.4+ MB
# of countries:  1
# of unique user statuses:  2


In [33]:
Q_kenya_df.to_csv('questions_kenya_df.csv', index=False)

In [34]:
## subset country datasets
Q_uganda_df = Q_df[(Q_df['question_user_country_code'] == 'ug') & ((Q_df['question_user_status'] == 'live') | (Q_df['question_user_status'] == 'zombie'))]
Q_tanzania_df = Q_df[(Q_df['question_user_country_code'] == 'tz') & ((Q_df['question_user_status'] == 'live') | (Q_df['question_user_status'] == 'zombie'))]
Q_gambia_df = Q_df[(Q_df['question_user_country_code'] == 'gb') & ((Q_df['question_user_status'] == 'live') | (Q_df['question_user_status'] == 'zombie'))]

In [35]:
#save as CSV files
Q_kenya_df.to_csv('questions_kenya_df.csv', index=False)
Q_tanzania_df.to_csv('questions_tanzania_df.csv', index=False)
Q_uganda_df.to_csv('questions_uganda_df.csv', index=False)
Q_gambia_df.to_csv('questions_gambia_df.csv', index=False)

In [37]:
#parquet the country files
Q_kenya_df.to_parquet("questions_kenya_df.parquet")
Q_uganda_df.to_parquet("questions_uganda_df.parquet")
Q_tanzania_df.to_parquet("questions_tanzania_df.parquet")
Q_gambia_df.to_parquet("questions_gambia_df.parquet")